# 17. Convert Detection Log to GeoJSON

Given a detection log from applying the model to dashcam images in a survey area (filtered or unfiltered) align the detections to the OpenStreetMap data and produce geojson files to map and compare the output

## Configuration

Any configuration that is required to run this notebook can be customized in the next cell

In [1]:
# Name of the subdirectory containing dashcam footage for an area, split into frame images in a
# "split" subdirectory, with an associated "metadata.csv"
# This subdirectory is assumed to be in the 'data_sources' directory
import_directory = 'dashcam_tour_mount_eliza'

# Which "locality" do we wish to process?
locality = 'Mount Eliza'

# We will sample the middle of each intersection, but we can also sample a
# "margin" around the intersection, at 10m intervals.
# E.g. if we set this to "20" then we will sample points at:
#    -20m, -10m, 0m, 10m, and 20m
# from the centre of the intersection, along the assumed bearing of the road
# Used here just to get the right filename
margin = 20

# Name of the detection log filename
filename_in  = 'detection_log_filtered.csv'

# Name of the output detection log that has been aligned to the OpenStreetMap data
filename_out = 'detection_log_filtered_snap.csv'

## Code

In [2]:
# General imports
import os
import sys

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import osm_gsv_utils.osm_walker as osm_walker

In [3]:
# Derive paths for configuration

locality_clean = locality.replace(' ', '_')

filename_main       = os.path.join(module_path_root, 'data_sources', 'Locality_' + locality_clean + '.osm')
filename_margin     = os.path.join(module_path_root, 'data_sources', 'Locality_' + locality_clean + '_margin.osm')
locality_margin     = '{0:s}_{1:d}m'.format(locality_clean, margin)

detections_dir = os.path.join(module_path_root, 'data_sources', import_directory, 'detections')

path_in  = os.path.join(detections_dir, filename_in)
path_out = os.path.join(detections_dir, filename_out)

In [4]:
# Load OSM data
walker = osm_walker(filename_margin, filename_margin, verbose=False)

  0%|          | 0/4274 [00:00<?, ?it/s]

  0%|          | 0/4274 [00:00<?, ?it/s]

In [5]:
# Compile ways into continuous roads
walker.link_way_sections(verbose=False)

In [6]:
# Snap detections to nearest intersections
walker.snap_detection_log(path_in, path_out)

  0%|          | 0/523 [00:00<?, ?it/s]

In [7]:
# Load the snapped detection log
walker.load_detection_log(path_out)

In [8]:
# Write GeoJSON files, including comparison files
walker.write_geojsons(locality_margin, detections_dir, intersection_skip_limit=2, infer_ends=False, verbose=False)

Writing hit, feature count: 14
Writing to: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections\hit.geojson
Writing tag, feature count: 13
Writing to: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections\tag.geojson
Writing both, feature count: 7
Writing to: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections\both.geojson
Writing either, feature count: 20
Writing to: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections\either.geojson
Writing hit_only, feature count: 8
Writing to: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections\hit_only.geojson
Writing tag_only, feature count: 8
Writing to: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections\tag_only.geojson
hit     : Total distance   11386.70m
tag     : Total distance   16900.42m
both    : Total distance    6749.93m
either  : Total distance   21537.19m
hit_only: Total distance    4588.17m
tag_only: Total distan

## TODO

This comparison against OSM is not useful yet.  The OSM data will include many roads that weren't included in the footage.  We really only want to run a comparison for what was covered.